# CostPro Quarterly Sales Metrics Dashboard

As a Junior Data Scientist at a online retailer, CostPro, you have the opportunity to work on a project that will have a major impact on the company. Your goal is to create a metrics dashboard that will provide a clear overview of the business' sales metrics, which your leadership team can use to guide business decisions. You'll use measures of central tendency and variability to help CostPro better understand their quarterly sales and visualize the likelihood that a customer will arrive on the website in a given time interval.

## Project Setup

First, we need to download and import all of the dependencies that we will need for the project

### Dependencies

In [ ]:
# Install all required dependencies for the project
!pip install -qqq numpy pandas seaborn matplotlib gdown plotly scipy
!pip install -qqq --force-reinstall protobuf==3.20 ##We need this to avoid version incompatibilities between packages

In [ ]:
# Import dependencies
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import shutil
import gdown
from google.colab import files

# [TO BE IMPLEMENTED]
# Feel free to add any other imports that you need to write your own custom metrics

## Data

[Online Retail](https://archive.ics.uci.edu/ml/datasets/online+retail) is a collection of roughly 400,000 records from an international online retail dataset. For this project, we will use a slightly modified (cleaned up) version of this dataset as the sales data for CostPro.

### Download the Dataset

In [ ]:
# Download the file from co:rise google drive
file_name = "online_retail.csv"
unique_id = "16HZKULqv2sX6AMqi4A8ndQ9ULeCLo4Qt"
gdown.download(id=unique_id, output=file_name)

### Clean & Prepare the Dataset

Now we'll import the data as a pandas dataframe. The code below will show you the first 5 rows of the dataframe, but feel free to explore the data further!

In [ ]:
# import data and show first 5 rows
data = pd.read_csv(file_name)
data.head()

## Project Jumpstart

### Explore Basic Statistics You Get Out of the Box with Pandas

In [ ]:
# check record count and column count
data.shape

In [ ]:
# review basic information about the dataset including column names, data types, non-null counts, and memory usage
data.info()

In [ ]:
# check the number of missing values in each column
data.isnull().sum(axis=0)

On the job, you'll need to explore if there are specific patterns to the missing values. For the sake of expediency and simplicity in this project, we'll drop them.

In [ ]:
# Drop missing values and validate that there are no missing values after the fix
data = data.dropna().reset_index(drop=True)
data.isnull().sum(axis=0)

In [ ]:
# Check the number of unique values in each column
data.nunique()

In [ ]:
# Use pandas describe() method to get a summary of the numerical columns
basic_stats = pd.DataFrame(data.describe())
basic_stats

Now we need to create columns for Year, Month, Day, and Quarter so that your leadership team at CostPro can look at statistics for specific time periods in your dashboard.

In [ ]:
# Format date columns
# Note: the pandas to_datetime function is being deprecated in the near future
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

data["Year"] = data["InvoiceDate"].dt.year
data["Month"] = data["InvoiceDate"].dt.month
data["Day"] = data["InvoiceDate"].dt.day
data["Quarter"] = data["InvoiceDate"].dt.quarter

### Clean the Data

Check the basic stats on the numeric data again, what do you notice?

In [ ]:
basic_stats

Look at the quantity minimum! `-80995.000000`

Let's take a look at the dataset again to figure out what's going on!

In [ ]:
# Review records where the quantity is less than 0
data[data['Quantity'] < 0].head()

After closer examination, we can see that both the quantities and pricing are negative. We can also see that the Invoice Numbers are prepended with a `C`.

A quick glance at the data docs tells us that "InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation."

In [ ]:
# Create a function to remove cancelled orders
def remove_cancelled_orders(data):
    # Create a subset of the data that only has cancelled invoices
    cancelled_invoices = data[data['InvoiceNo'].str.contains('C')]

    # Create a new column that contains the original invoice number
    cancelled_invoices['OriginalInvoiceNo'] = cancelled_invoices['InvoiceNo'].str.replace('C', '')

    # Filter the dataset to only include invoices that haven't been cancelled
    data = data[~data['InvoiceNo'].str.contains('C')]

    # Filter the data by the original invoice number on the cancelled invoices
    return data[~data['InvoiceNo'].isin(cancelled_invoices['OriginalInvoiceNo'])]

data = remove_cancelled_orders(data)

data.head()

# Review the data again to ensure that the cancelled orders have been removed
assert data[data['InvoiceNo'].str.contains('C')].shape[0] == 0


Great! That's looking much better! Now the minimum quantity is 1.

On the job, we'd definitely want to dig into what products get returned and at what rate and factor returns into total sales. For the sake of expediency in this project, we're going to ignore cancellations and returns.

### Create Useful Aggregations

In [ ]:
# Create a column that represents the total price of each item
data['TotalPrice'] = data['Quantity'] * data['UnitPrice']

In [ ]:
# Create a dataframe that represents the total value of each order
total_value_per_order = pd.DataFrame(data.groupby('InvoiceNo')['TotalPrice'].sum())
total_value_per_order.head()

### Measures of Central Tendency

#### Mean

In [ ]:
# Calculate the average total value of each order
total_value_per_order['TotalPrice'].mean()

#### Median

In [ ]:
# Calculate the median total value of each order
total_value_per_order['TotalPrice'].median()

Since there's a sizeable difference between the mean and the median, we know the data is skewed. This means we should probably display the median on the dashboard and also visualize the data distribution to highlight the skew.

#### Inter-Quartile Range
Use Interquartile range to help you understand the central tendencies and range of the data.

In [ ]:
# visualize the interquartile range of the total price as a boxplot
sns.boxplot(total_value_per_order['TotalPrice'])

In [ ]:
# Remove outliers that are more than 1.5 times the interquartile range
q1 = total_value_per_order['TotalPrice'].quantile(0.25)
q3 = total_value_per_order['TotalPrice'].quantile(0.75)
iqr = q3 - q1
total_value_per_order = total_value_per_order[(total_value_per_order['TotalPrice'] >= q1 - 1.5 * iqr) & (total_value_per_order['TotalPrice'] <= q3 + 1.5 * iqr)]

In [ ]:
# Check the boxplot again
sns.boxplot(total_value_per_order['TotalPrice'])

In [ ]:
# Reset the index to make it easier to filter on InvoiceNo
total_value_per_order = total_value_per_order.reset_index(level=0)

In [ ]:
# Filter the data by the dataframe where we've removed outliers
data = data[data['InvoiceNo'].isin(total_value_per_order['InvoiceNo'])]
data.head()

Now we're seeing a dataset that looks a lot more like what we'd expect to see for purchase data, however, there is still a lot of variability.

### Measures of Variability

#### Standard Deviation

In [ ]:
# Calculate the standard deviation of the total value of each order
total_value_per_order['TotalPrice'].std()

In [ ]:
# Visualize the distribution of the total order value
sns.displot(total_value_per_order['TotalPrice'], kde=True)

#### Variance

In [ ]:
# Calculate the variance of the total value of each order
total_value_per_order['TotalPrice'].var()

#### Maximum

In [ ]:
# Calculate the maximum order value
maximum_order_value = round(total_value_per_order['TotalPrice'].max(), 2)
maximum_order_value

#### Minimum

In [ ]:
# Calculate the minimum order value
minimum_order_value = round(total_value_per_order['TotalPrice'].min(), 2)
minimum_order_value

#### Range
What is the new range of this dataset after removing outliers?

In [ ]:
# Calculate the range of the total value of each order
maximum_order_value - minimum_order_value

## Your Turn! Apply This Week's Concepts

You'll be writing these tasks inside python functions. This helps you in many ways:

- easier for you to debug
- easier for you to test
- allows you to set up metrics for your dashboard without re-writing this code later!

### Filter Data By Year and Quarter

Here's an example, you get the first one for free!

In [ ]:
# Write a function to filter a pandas dataframe by year and quarter
def filter_by_year_and_quarter(df, year, quarter):

    # Filter the dataframe by the year and quarter
    filtered_df = df[(df['Year'] == year) & (df['Quarter'] == quarter)]

    # Return the filtered dataframe
    if filtered_df.shape[0] > 0:
        return filtered_df
    else:
        assert filtered_df.shape[0] > 0, "There are no records for the specified year and quarter"

In [ ]:
# Create a dataframe that only contains records for 2022 Q4
print(filter_by_year_and_quarter(data, 2022, 4).head(5))

# Write a test to validate the function
assert filter_by_year_and_quarter(data, 2022, 4)['Year'].unique()[0] == 2022, "There should only be records for 2022"
assert filter_by_year_and_quarter(data, 2022, 4)['Quarter'].unique()[0] == 4, "There should be records for Q4"

### Calculate Total Sales (Sum)

In [ ]:
def total_sales(data: pd.DataFrame, year: int, quarter: int) -> float:
    """
    Calculate the total sales for a given year and quarter
    :param data: The dataframe containing the data
    :param year: The year to calculate the total sales for
    :param quarter: The quarter to calculate the total sales for
    :return: The total sales for the given year and quarter
    """
    # Filter the data by the given year and quarter
    data = filter_by_year_and_quarter(data, year, quarter)

    # Calculate the total sales
    return <YOUR CODE HERE>

In [ ]:
# Calculate the total sales for 2022 Q4
print(f'The total sales for 2022 Q4 are: ${total_sales(data, 2022, 4)}')

### How Many Customers Are There Per Day? (Mean)

In [ ]:
# Write a function to calculate mean number of customers per day
def mean_customers_per_day(data: pd.DataFrame, year: int, quarter: int) -> float:
    """
    Calculate the mean number of customers per day for a given year and quarter
    :param data: The dataframe containing the data
    :param year: The year to calculate the mean number of customers per day for
    :param quarter: The quarter to calculate the mean number of customers per day for
    :return: The mean number of customers per day for the given year and quarter
    """
    # Filter the data by the given year and quarter
    data = filter_by_year_and_quarter(data, year, quarter)

    # Calculate the mean number of customers per day
    return <YOUR CODE HERE>

In [ ]:
# Calculate the mean number of customers per day for 2022 Q4
print(f'The mean number of customers per day for 2022 Q4 is: {mean_customers_per_day(data, 2022, 4)}')

# Write a test to validate the mean number of customers per day function
assert <YOUR CODE HERE>, "your message here"

### How Much Do Customers Spend Per Order? (Median)

In [ ]:
# Write a function to calculate the median order value for a given year and quarter
def median_order_value(data: pd.DataFrame, year: int, quarter: int) -> float:
    """
    Calculate the median order value for a given year and quarter
    :param data: The dataframe containing the data
    :param year: The year to calculate the median order value for
    :param quarter: The quarter to calculate the median order value for
    :return: The median order value for the given year and quarter
    """
    # Filter the data by the given year and quarter
    data = filter_by_year_and_quarter(data, year, quarter)

    # Group data by order number and calculate the order total
    <YOUR CODE HERE>

    # Calculate the median order value
    return <YOUR CODE HERE>

In [ ]:
# Calculate the median order value for 2022 Q4
print(f'The median order value for 2022 Q4 is: ${median_order_value(data, 2022, 4)}')

In [ ]:
# Write a test to validate the median order value function
assert <YOUR CODE HERE>

### How Variable Are Order Totals?

In [ ]:
# Write a function to calculate the order value variability for a given year and quarter
def order_value_variability(data: pd.DataFrame, year: int, quarter: int) -> float:
    """
    Calculate the order value variability for a given year and quarter
    :param data: The dataframe containing the data
    :param year: The year to calculate the order value variability for
    :param quarter: The quarter to calculate the order value variability for
    :return: The order value variability for the given year and quarter
    """
    # Filter the data by the given year and quarter
    data = filter_by_year_and_quarter(data, year, quarter)

    # Group data by order number and calculate the order total
    <YOUR CODE HERE>

    # Calculate the order value variability
    return <YOUR CODE HERE>

In [ ]:
# Print out the order value variability for 2022 Q4
print(f'The standard deviation of order value for 2022 Q4 is: ${order_value_variability(data, 2022, 4)}')

In [ ]:
# Write a test to validate the order value variability function
assert <YOUR CODE HERE>

### What is the Most Popular Product? (Mode - Part 1)

In [ ]:
# Write a function to find the most popular product for a given year and quarter
def most_popular_product(data: pd.DataFrame, year: int, quarter: int) -> str:
    """
    Find the most popular product for a given year and quarter
    :param data: The dataframe containing the data
    :param year: The year to find the most popular product for
    :param quarter: The quarter to find the most popular product for
    :return: The most popular product for the given year and quarter
    """
    # Filter the data by the given year and quarter
    data = filter_by_year_and_quarter(data, year, quarter)

    # Find the most popular product
    return <YOUR CODE HERE>

In [ ]:
# Find the most popular product for 2022 Q4
print(f'The most popular product for 2022 Q4 is: {most_popular_product(data, 2022, 4)}')

# Write a test to validate the most popular product function
assert <YOUR CODE HERE>

### What is the Most Profitable Product? (Mode - Part 2)

In [ ]:
# Write a function to find the most profitable product for a given year and quarter
def most_profitable_product(data: pd.DataFrame, year: int, quarter: int) -> str:
    """
    Find the most profitable product for a given year and quarter
    :param data: The dataframe containing the data
    :param year: The year to find the most profitable product for
    :param quarter: The quarter to find the most profitable product for
    :return: The most profitable product for the given year and quarter
    """
    # Filter the data by the given year and quarter
    data = filter_by_year_and_quarter(data, year, quarter)

    # Find the most profitable product
    return <YOUR CODE HERE>

In [ ]:
# Find the most profitable product for 2022 Q4
print(f'The most profitable product for 2022 Q4 is: {most_profitable_product(data, 2022, 4)}')

# Write a test to validate the most profitable product function
assert <YOUR CODE HERE>

## Your Turn: Build Custom Metrics (Optional)

Here's an example of a metric that you'd expect to see on sales dashboard: percentage of returning customers. Use this example as a template to write your own metric!

### What Percentage of Customers Become Return Customers?

In [ ]:
def customer_retention_rate(data: pd.DataFrame, year: int, quarter: int) -> float:
    """
    Calculate the customer retention rate for a given year and quarter
    :param data: The dataframe containing the data
    :param year: The year to calculate the customer retention rate for
    :param quarter: The quarter to calculate the customer retention rate for
    :return: The customer retention rate for the given year and quarter
    """
    # Filter the data by the given year and quarter
    data = filter_by_year_and_quarter(data, year, quarter)

    # Calculate the number of unique customers
    num_customers = <YOUR CODE HERE>

    # Calculate the number of repeat customers
    num_repeat_customers = <YOUR CODE HERE>

    # Calculate the customer retention rate
    return <YOUR CODE HERE>

In [ ]:
# Calculate the customer retention rate for 2022 Q4
print(f'The customer retention rate for 2022 Q4 is: {customer_retention_rate(data, 2022, 4)}%')

# Write tests to validate the customer retention rate function
assert <YOUR CODE HERE>

## Make a Dashboard for Your Portfolio!

Optional, but ***Highly Encouraged***, Since You've Already Written the Code!

To bring our analysis to life we'll be using a toolkit known as Jupyter Widgets, alongside an interactive plot creation library called Plotly.

At this stage we don't anticipate that you're necessarily familiar with the ins and outs of how Jupyter Widgets function, but the code and links below should help you get started.

You're welcome to modify it as you please. In fact, data visualization is a crucial component of the data science toolkit, so if you're so inclined, give it a try! Some excellent starting points for learning about Jupyter Widgets can be found here, [here](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Basics.html), [here](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html), and [here](https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916)!

One potential place to start - can you come up with a better plot type for the data to show outliers? Have a scroll through what the plotly.express library can do [here](https://plotly.com/python/plotly-express/) for inspiration

In [ ]:
import pandas as pd
import plotly.express as px
from ipywidgets import interact, Select, Output, link
from IPython.display import display, clear_output

from google.colab import output
output.enable_custom_widget_manager()

# Create a copy of the data we've processed,
#  in case we want to modify it some more
df = data.copy(deep=True)

# Create an output widget
out = Output()

# Define function to execute on change of filter
def metrics_and_visualization(year, quarter):
  # Filter the data for the selected year and quarter
  filtered_df = filter_by_year_and_quarter(df, year, quarter)

  # Sample our data, to make the visualisation less computionationally intensive
  filtered_df = filtered_df.sample(frac=0.1, random_state=42)

  out.clear_output(wait=True)
  with out:
    # Display the metrics
    print(f"Q{quarter} Total sales: ${total_sales(filtered_df, year, quarter)}")
    print(f"Q{quarter} Mean Number of Customers Per Day: {mean_customers_per_day(filtered_df, year, quarter)}")
    print(f"Q{quarter} Median Order Value: ${median_order_value(filtered_df, year, quarter)}")
    print(f"Q{quarter} Standard Deviation of Order Values: ${order_value_variability(filtered_df, year, quarter)}")
    print(f"Q{quarter} Most Popular Product: {most_popular_product(filtered_df, year, quarter)}")
    print(f"Q{quarter} Most Profitable Product: {most_profitable_product(filtered_df, year, quarter)}")
    print(f"Q{quarter} Customer retention rate: {customer_retention_rate(filtered_df, year, quarter)}%")

    # Create a scatterplot of unit prices for the selected quarter
    fig = px.scatter(
        filtered_df,
        x="UnitPrice",
        color="UnitPrice",
        width=800,
        height=400
    )
    fig.show()

# Create a selection widget to choose the quarter
year_widget = Select(options=df["Year"].unique().tolist(), description='Year:')
quarter_widget = Select(options=df["Quarter"].unique().tolist(), description='Quarter:')

# Display widgets
display(year_widget, quarter_widget, out)

# Define function to update quarters widget based on the selected year
def update_quarters(change):
    year = change['new']
    quarters = df[df["Year"] == year]["Quarter"].unique().tolist()
    quarter_widget.options = quarters
    quarter_widget.value = quarters[0] if quarters else None

# Bind the function to changes in the year widget
year_widget.observe(update_quarters, names='value')

# Bind the metrics_and_visualization function to changes in the widgets
def on_change(change):
    metrics_and_visualization(year_widget.value, quarter_widget.value)

quarter_widget.observe(on_change, names='value')

# Call function once to update quarters and display initial data
update_quarters({'new': year_widget.value})
metrics_and_visualization(year_widget.value, quarter_widget.value)

We often want to share visualisations like this with our colleagues and let them play around but without exposing them to all the analysis code we took to get there. Luckily we can easily do this in Colab!
* go to `Edit` -> `Select All Cells`
* go to `View` -> `Show/hide code`

You can also go through and collapse individual sections that you don't want to share (e.g. the bit at the top where we installed some python libraries).

You can then share a link to the notebook (`Share` -> `Copy Link`, setting the `General Access` field appropriately) and then anyone you share the link with can open the notebook, run through it, and see your analysis!

#### 🚀 You Did It!!!

Congratulations, you've completed your first assignment in Applied Statistics for Data Science. If you deployed a streamlit app, please be sure to share the link when you submit your assignment for the week.